<a href="https://colab.research.google.com/github/Pontes-y/fiap/blob/yp_fiap_ano2_fase4/fiap_ano2_fase_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fase 7\
Notebook para importar as imagens de URLs em um CSV para o bucket na Azure

In [ ]:
# Biblioteca azure-storage-blob
!pip install azure-storage-blob

In [66]:
import os
import pandas as pd
import subprocess
from google.colab import userdata
from urllib.parse import urlparse
from datetime import datetime, timedelta
from azure.storage.blob import BlobServiceClient, generate_container_sas, AccountSasPermissions, ResourceTypes

In [67]:
# Arquivo CSV com as URLs
df = pd.read_csv('BX-Books.csv')

In [68]:
# Configurações da conta da Azure com segredos do Colab
account_url = userdata.get('account_url')
account_name = userdata.get('account_name')
account_key = userdata.get('account_key')

In [69]:
# Ciando um SAS tokens
sas_token = generate_container_sas(
    account_name=account_name,
    container_name="bx-images",
    account_key=account_key,
    permission=AccountSasPermissions(read=True, write=True, delete=True),
    expiry=datetime.utcnow() + timedelta(days=1),
    resource_types=ResourceTypes(container=True, object=True)
)

In [70]:
# Conectando ao Blob Storage
blob_service_client = BlobServiceClient(account_url=account_url, credential=sas_token)
container_name = "bx-images"
container_client = blob_service_client.get_container_client(container_name)

In [72]:
# Função de download da imagem e upload para o Blob Storage
def download_and_upload_image(url):
    file_name = os.path.basename(urlparse(url).path)
    blob_prefix = f"Image-URL-S/{file_name}"
    subprocess.call(['curl', '-o', file_name, url])
    blob_client = container_client.get_blob_client(blob_prefix)
    with open(file_name, 'rb') as f:
        blob_client.upload_blob(f)

In [73]:
for index, row in df.iterrows():
    url = row['Image-URL-S']
    if url:
        download_and_upload_image(url)